In [ ]:
Questions 1 Ans :

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

def search_amazon(product_name):
    # Set the path to your webdriver. Download the webdriver for your browser.
    # For Chrome: https://sites.google.com/a/chromium.org/chromedriver/downloads
    webdriver_path = '/path/to/chromedriver'

    # Create a new instance of the Chrome driver
    driver = webdriver.Chrome(executable_path=webdriver_path)

    # Navigate to Amazon.in
    driver.get("https://www.amazon.in/")

    # Find the search input field and enter the product name
    search_box = driver.find_element("id", "twotabsearchtextbox")
    search_box.send_keys(product_name)
    search_box.send_keys(Keys.RETURN)

    # Wait for the results to load
    driver.implicitly_wait(5)

    # Print the titles of the search results
    results = driver.find_elements_by_css_selector('.s-title-instructions h2 a')
    for index, result in enumerate(results, start=1):
        print(f"{index}. {result.text}\n{result.get_attribute('href')}\n")

    # Close the browser window
    driver.quit()

if __name__ == "__main__":
    # Take user input for the product to be searched
    product_to_search = input("Enter the product to be searched on Amazon: ")

    # Perform the search
    search_amazon(product_to_search)

ModuleNotFoundError: No module named 'selenium'

In [ ]:
Questions 2 Ans :

In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

def scrape_amazon(product_name):
    # Set the path to your webdriver. Download the webdriver for your browser.
    # For Chrome: https://sites.google.com/a/chromium.org/chromedriver/downloads
    webdriver_path = '/path/to/chromedriver'

    # Create a new instance of the Chrome driver
    driver = webdriver.Chrome(executable_path=webdriver_path)

    # Navigate to Amazon.in
    driver.get("https://www.amazon.in/")

    # Find the search input field and enter the product name
    search_box = driver.find_element("id", "twotabsearchtextbox")
    search_box.send_keys(product_name)
    search_box.send_keys(Keys.RETURN)

    # Wait for the results to load
    driver.implicitly_wait(5)

    # Initialize data lists
    brand_names = []
    product_names = []
    prices = []
    return_exchange = []
    expected_delivery = []
    availability = []
    product_urls = []

    # Loop through the first 3 pages of search results
    for page in range(3):
        # Print the titles of the search results on the current page
        results = driver.find_elements_by_css_selector('.s-title-instructions h2 a')
        for result in results:
            # Click on each product to get detailed information
            result.click()

            # Wait for the product details page to load
            driver.implicitly_wait(5)

            # Scrape details from the product page
            brand_name = driver.find_element_by_id("bylineInfo").text.strip() if driver.find_element_by_id("bylineInfo").text.strip() else "-"
            product_name = driver.find_element_by_id("productTitle").text.strip() if driver.find_element_by_id("productTitle").text.strip() else "-"
            price = driver.find_element_by_id("priceblock_ourprice").text.strip() if driver.find_element_by_id("priceblock_ourprice").text.strip() else "-"
            return_exchange_info = driver.find_element_by_xpath("//span[contains(text(),'Return policy')]/following-sibling::span").text.strip() if driver.find_element_by_xpath("//span[contains(text(),'Return policy')]/following-sibling::span").text.strip() else "-"
            expected_delivery_info = driver.find_element_by_xpath("//div[@id='ddmDeliveryMessage']/b").text.strip() if driver.find_element_by_xpath("//div[@id='ddmDeliveryMessage']/b").text.strip() else "-"
            availability_info = driver.find_element_by_id("availability").text.strip() if driver.find_element_by_id("availability").text.strip() else "-"
            product_url = driver.current_url.strip() if driver.current_url.strip() else "-"

            # Append details to the lists
            brand_names.append(brand_name)
            product_names.append(product_name)
            prices.append(price)
            return_exchange.append(return_exchange_info)
            expected_delivery.append(expected_delivery_info)
            availability.append(availability_info)
            product_urls.append(product_url)

            # Go back to the search results page
            driver.back()

        # Navigate to the next page of search results
        next_page_button = driver.find_element_by_partial_link_text("Next")
        if next_page_button:
            next_page_button.click()
            driver.implicitly_wait(5)
        else:
            break

    # Create a DataFrame with the scraped data
    data = {
        "Brand Name": brand_names,
        "Name of the Product": product_names,
        "Price": prices,
        "Return/Exchange": return_exchange,
        "Expected Delivery": expected_delivery,
        "Availability": availability,
        "Product URL": product_urls,
    }
    df = pd.DataFrame(data)

    # Save the DataFrame to a CSV file
    df.to_csv(f"{product_name}_search_results.csv", index=False)

    # Close the browser window
    driver.quit()

if __name__ == "__main__":
    # Take user input for the product to be searched
    product_to_search = input("Enter the product to be searched on Amazon: ")

    # Perform the search and scraping
    scrape_amazon(product_to_search)

ModuleNotFoundError: No module named 'selenium'

In [ ]:
Questions 3 Ans :

In [4]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import os

def save_images(keyword, image_urls):
    # Create a directory for each keyword
    directory = f"{keyword}_images"
    os.makedirs(directory, exist_ok=True)

    # Download and save each image
    for i, url in enumerate(image_urls):
        response = requests.get(url, stream=True)
        with open(os.path.join(directory, f"{keyword}_{i+1}.jpg"), 'wb') as file:
            for chunk in response.iter_content(chunk_size=128):
                file.write(chunk)

def scrape_images(keyword):
    # Set the path to your webdriver. Download the webdriver for your browser.
    # For Chrome: https://sites.google.com/a/chromium.org/chromedriver/downloads
    webdriver_path = '/path/to/chromedriver'

    # Create a new instance of the Chrome driver
    driver = webdriver.Chrome(executable_path=webdriver_path)

    # Navigate to Google Images
    driver.get("https://www.google.com/imghp")

    # Find the search bar and enter the keyword
    search_bar = driver.find_element("name", "q")
    search_bar.send_keys(keyword)
    search_bar.send_keys(Keys.RETURN)

    # Wait for the results to load
    time.sleep(3)

    # Scroll down to load more images
    for _ in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    # Get the page source after scrolling
    page_source = driver.page_source

    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')

    # Find image elements and extract URLs
    image_elements = soup.find_all('img', class_='rg_i')
    image_urls = [element['src'] for element in image_elements]

    # Save 10 images for the specified keyword
    save_images(keyword, image_urls[:10])

    # Close the browser window
    driver.quit()

if __name__ == "__main__":
    # List of keywords to search for
    keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']

    for keyword in keywords:
        print(f"Scraping images for keyword: {keyword}")
        scrape_images(keyword)
        

ModuleNotFoundError: No module named 'selenium'

In [ ]:
Questions 4 Ans :

In [5]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

def scrape_flipkart_smartphones(product_name):
    # Set the path to your webdriver. Download the webdriver for your browser.
    # For Chrome: https://sites.google.com/a/chromium.org/chromedriver/downloads
    webdriver_path = '/path/to/chromedriver'

    # Create a new instance of the Chrome driver
    driver = webdriver.Chrome(executable_path=webdriver_path)

    # Navigate to Flipkart
    driver.get("https://www.flipkart.com/")

    # Find the search bar and enter the product name
    search_bar = driver.find_element("name", "q")
    search_bar.send_keys(product_name)
    search_bar.send_keys(Keys.RETURN)

    # Wait for the results to load
    driver.implicitly_wait(5)

    # Get the page source after searching
    page_source = driver.page_source

    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')

    # Find smartphone elements on the first page
    smartphone_elements = soup.find_all('div', class_='_1AtVbE')

    # Initialize data lists
    brand_names = []
    smartphone_names = []
    colors = []
    rams = []
    storages = []
    primary_cameras = []
    secondary_cameras = []
    display_sizes = []
    battery_capacities = []
    prices = []
    product_urls = []

    # Extract details from each smartphone element
    for element in smartphone_elements:
        # Brand and smartphone name
        brand_name = element.find('div', class_='_4rR01T').text.strip() if element.find('div', class_='_4rR01T') else "-"
        smartphone_name = element.find('a', class_='IRpwTa').text.strip() if element.find('a', class_='IRpwTa') else "-"

        # Other details
        details = element.find_all('li', class_='rgWa7D')
        color = details[0].text.strip() if details and len(details) > 0 else "-"
        ram = details[1].text.strip() if details and len(details) > 1 else "-"
        storage = details[2].text.strip() if details and len(details) > 2 else "-"
        primary_camera = details[3].text.strip() if details and len(details) > 3 else "-"
        secondary_camera = details[4].text.strip() if details and len(details) > 4 else "-"
        display_size = details[5].text.strip() if details and len(details) > 5 else "-"
        battery_capacity = details[6].text.strip() if details and len(details) > 6 else "-"
        price = element.find('div', class_='_30jeq3').text.strip() if element.find('div', class_='_30jeq3') else "-"
        product_url = "https://www.flipkart.com" + element.find('a', class_='IRpwTa')['href'] if element.find('a', class_='IRpwTa') else "-"

        # Append details to the lists
        brand_names.append(brand_name)
        smartphone_names.append(smartphone_name)
        colors.append(color)
        rams.append(ram)
        storages.append(storage)
        primary_cameras.append(primary_camera)
        secondary_cameras.append(secondary_camera)
        display_sizes.append(display_size)
        battery_capacities.append(battery_capacity)
        prices.append(price)
        product_urls.append(product_url)

    # Create a DataFrame with the scraped data
    data = {
        "Brand Name": brand_names,
        "Smartphone Name": smartphone_names,
        "Colour": colors,
        "RAM": rams,
        "Storage(ROM)": storages,
        "Primary Camera": primary_cameras,
        "Secondary Camera": secondary_cameras,
        "Display Size": display_sizes,
        "Battery Capacity": battery_capacities,
        "Price": prices,
        "Product URL": product_urls,
    }
    df = pd.DataFrame(data)

    # Save the DataFrame to a CSV file
    df.to_csv(f"{product_name}_search_results.csv", index=False)

    # Close the browser window
    driver.quit()

if __name__ == "__main__":
    # Take user input for the smartphone to be searched
    product_to_search = input("Enter the smartphone to be searched on Flipkart: ")

    # Perform the search and scraping
    scrape_flipkart_smartphones(product_to_search)

ModuleNotFoundError: No module named 'selenium'

In [ ]:
Question 5 Ans :

In [7]:
pip install googlemaps
import googlemaps

def get_coordinates(api_key, city_name):
    # Create a Google Maps client
    gmaps = googlemaps.Client(key=api_key)

    # Geocode the city
    geocode_result = gmaps.geocode(city_name)

    # Extract the latitude and longitude
    if geocode_result and 'geometry' in geocode_result[0] and 'location' in geocode_result[0]['geometry']:
        location = geocode_result[0]['geometry']['location']
        latitude = location['lat']
        longitude = location['lng']

        return latitude, longitude
    else:
        return None

if __name__ == "__main__":
    # Replace 'YOUR_API_KEY' with your actual Google Maps API key
    api_key = 'YOUR_API_KEY'

    # Take user input for the city to be searched
    city_name = input("Enter the city name: ")

    # Get the geospatial coordinates
    coordinates = get_coordinates(api_key, city_name)

    if coordinates:
        print(f"Geospatial Coordinates (Latitude, Longitude) for {city_name}: {coordinates}")
    else:
        print(f"Unable to retrieve geospatial coordinates for {city_name}.")

SyntaxError: invalid syntax (4124411766.py, line 1)

In [ ]:
Question 6 Ans :

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_digit_gaming_laptops():
    url = "https://www.digit.in/top-products/best-gaming-laptops-40.html"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        laptops = soup.find_all('div', class_='TopNumbeHeading active sticky-footer')

        data = {
            "Name": [],
            "Processor": [],
            "RAM": [],
            "OS": [],
            "Storage": [],
            "Display": [],
            "Price": []
        }

        for laptop in laptops:
            name = laptop.find('div', class_='TopNumbeHeadingname').text.strip()
            processor = laptop.find('div', class_='TopNumbeHeadingpname').text.strip()
            ram = laptop.find('div', class_='TopNumbeHeadingnname').text.strip()
            os = laptop.find('div', class_='TopNumbeHeadingname').text.strip()
            storage = laptop.find('div', class_='TopNumbeHeadingname').text.strip()
            display = laptop.find('div', class_='TopNumbeHeadingname').text.strip()
            price = laptop.find('div', class_='TopNumbeHeadingprice').text.strip()

            data["Name"].append(name)
            data["Processor"].append(processor)
            data["RAM"].append(ram)
            data["OS"].append(os)
            data["Storage"].append(storage)
            data["Display"].append(display)
            data["Price"].append(price)

        df = pd.DataFrame(data)
        df.to_csv("gaming_laptops.csv", index=False)
        print("Data has been scraped and saved to 'gaming_laptops.csv'")
    else:
        print(f"Failed to retrieve data. Status Code: {response.status_code}")

if __name__ == "__main__":
    get_digit_gaming_laptops()

Data has been scraped and saved to 'gaming_laptops.csv'


In [ ]:
Qustions 7 Ans :

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_forbes_billionaires():
    url = "https://www.forbes.com/billionaires/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        billionaires = soup.find_all('div', class_='personListItem')

        data = {
            "Rank": [],
            "Name": [],
            "Net Worth": [],
            "Age": [],
            "Citizenship": [],
            "Source": [],
            "Industry": []
        }

        for billionaire in billionaires:
            rank = billionaire.find('div', class_='rank').text.strip()
            name = billionaire.find('div', class_='name').text.strip()
            net_worth = billionaire.find('div', class_='netWorth').text.strip()
            age = billionaire.find('div', class_='age').text.strip()
            citizenship = billionaire.find('div', class_='countryOfCitizenship').text.strip()
            source = billionaire.find('div', class_='source').text.strip()
            industry = billionaire.find('div', class_='category').text.strip()

            data["Rank"].append(rank)
            data["Name"].append(name)
            data["Net Worth"].append(net_worth)
            data["Age"].append(age)
            data["Citizenship"].append(citizenship)
            data["Source"].append(source)
            data["Industry"].append(industry)

        df = pd.DataFrame(data)
        df.to_csv("forbes_billionaires.csv", index=False)
        print("Data has been scraped and saved to 'forbes_billionaires.csv'")
    else:
        print(f"Failed to retrieve data. Status Code: {response.status_code}")

if __name__ == "__main__":
    scrape_forbes_billionaires()

Data has been scraped and saved to 'forbes_billionaires.csv'


In [ ]:
Questions 8 Ans :

In [11]:
from googleapiclient.discovery import build
import pandas as pd

def get_youtube_comments(api_key, video_id, max_results=500):
    youtube = build('youtube', 'v3', developerKey=api_key)

    # Get video details
    video_response = youtube.videos().list(
        part='snippet',
        id=video_id
    ).execute()

    video_title = video_response['items'][0]['snippet']['title']

    # Get comments
    comments = []
    next_page_token = None

    while len(comments) < max_results:
        comment_response = youtube.commentThreads().list(
            part='snippet,replies',
            videoId=video_id,
            maxResults=min(max_results - len(comments), 100),
            pageToken=next_page_token
        ).execute()

        for item in comment_response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments.append({
                'Comment': comment['textDisplay'],
                'UpvoteCount': comment['likeCount'],
                'Time': comment['publishedAt']
            })

        next_page_token = comment_response.get('nextPageToken')

        if not next_page_token:
            break

    df = pd.DataFrame(comments)
    df.to_csv(f'{video_title}_comments.csv', index=False)
    print(f"Data has been scraped and saved to '{video_title}_comments.csv'")

if __name__ == "__main__":
    # Replace 'YOUR_API_KEY' with your actual YouTube API key
    api_key = 'YOUR_API_KEY'

    # Replace 'VIDEO_ID' with the actual video ID you want to scrape comments from
    video_id = 'VIDEO_ID'

    # Get YouTube comments
    get_youtube_comments(api_key, video_id, max_results=500)

ModuleNotFoundError: No module named 'googleapiclient'

In [ ]:
Questions 9 Ans :

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_hostelworld_london():
    url = "https://www.hostelworld.com/s?q=London,%20England&country=England&city=London&type=city&id=3&from=2022-03-07&to=2022-03-14&guests=1&page=1"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        hostels = soup.find_all('div', class_='property-card')

        data = {
            "Hostel Name": [],
            "Distance from City Centre": [],
            "Ratings": [],
            "Total Reviews": [],
            "Overall Reviews": [],
            "Privates from Price": [],
            "Dorms from Price": [],
            "Facilities": [],
            "Property Description": []
        }

        for hostel in hostels:
            name = hostel.find('h2', class_='title').text.strip()
            distance = hostel.find('span', class_='description').text.strip()
            ratings = hostel.find('div', class_='score orange big').text.strip()
            total_reviews = hostel.find('div', class_='reviews').text.strip()
            overall_reviews = hostel.find('div', class_='reviews').text.strip()
            privates_price = hostel.find('div', class_='price-col privates-from').text.strip()
            dorms_price = hostel.find('div', class_='price-col dorms-from').text.strip()
            facilities = [facility.text.strip() for facility in hostel.find_all('div', class_='facilities-item')]
            description = hostel.find('div', class_='property-card-text').text.strip()

            data["Hostel Name"].append(name)
            data["Distance from City Centre"].append(distance)
            data["Ratings"].append(ratings)
            data["Total Reviews"].append(total_reviews)
            data["Overall Reviews"].append(overall_reviews)
            data["Privates from Price"].append(privates_price)
            data["Dorms from Price"].append(dorms_price)
            data["Facilities"].append(', '.join(facilities))
            data["Property Description"].append(description)

        df = pd.DataFrame(data)
        df.to_csv("hostelworld_london.csv", index=False)
        print("Data has been scraped and saved to 'hostelworld_london.csv'")
    else:
        print(f"Failed to retrieve data. Status Code: {response.status_code}")

if __name__ == "__main__":
    scrape_hostelworld_london()